In [20]:
/*
    Created by Indra Mahkota
*/

%use @gson.json
%use krangl
%use coroutines
%use @retrofit.json
%use @retrofit_gson_converter.json
%use @org_json.json

import kotlinx.coroutines.flow.Flow
import kotlinx.coroutines.flow.flow
import kotlinx.coroutines.flow.MutableStateFlow
import kotlinx.coroutines.flow.StateFlow
import kotlinx.coroutines.flow.collect
import kotlinx.coroutines.GlobalScope
import kotlinx.coroutines.launch

In [67]:
data class UserResponse(
    @SerializedName("incomplete_results")
    val incompleteResults: Boolean?,
    @SerializedName("items")
    val items: List<UserDto>,
    @SerializedName("total_count")
    val totalCount: Long?
)

data class UserDto(
    @SerializedName("id")
    val id: Long,
    @SerializedName("avatar_url")
    val avatarUrl: String?,
    @SerializedName("name")
    val name: String?,
    @SerializedName("login")
    val login: String?,
    @SerializedName("bio")
    val bio: String?,
    @SerializedName("followers")
    val followers: Int?,
    @SerializedName("following")
    val following: Int?,
    @SerializedName("location")
    val location: String?,
    @SerializedName("email")
    val email: String?
)

data class User(
    val id: Long,
    val avatarUrl: String,
    val name: String,
    val login: String,
    val bio: String,
    val followers: Int,
    val following: Int,
    val location: String,
    val email: String
)

In [71]:
fun UserDto.toUser() = User(
    id = id,
    avatarUrl = avatarUrl ?: "",
    name = name ?: "N/A",
    login = login ?: "N/A",
    bio = bio ?: "Biography not available",
    followers = followers ?: 0,
    following = following ?: 0,
    location = location ?: "N/A",
    email = email ?: "N/A"
)

fun List<UserDto>.toListUser() =
    this.map {
        it.toUser()
    }

In [68]:
interface GithubService {
    @JvmSuppressWildcards
    @GET("search/users")
    suspend fun searchUsers(
        @QueryMap params: Map<String, Any>
    ): Response<UserResponse>

    @GET("users/{id}")
    suspend fun getUser(
        @Path("id") username: String
    ): Response<UserDto>
}

In [69]:
val retrofit = Retrofit
        .Builder().apply {
            baseUrl("https://api.github.com/")
            addConverterFactory(GsonConverterFactory.create(GsonBuilder().create()))
        }.build()
        
val service = retrofit.create(GithubService::class.java)

In [73]:
suspend fun searchUsers(params: Map<String, Any>): UserResponse? {
    return try {
        val response = service.searchUsers(params)
        if(response.isSuccessful)
            response.body()
        else null
    } catch (e: Exception) {
        println(e.message ?: "Unknown error")
        null
    }
}

suspend fun getUser(username: String): UserDto? {
    return try {
        val response = service.getUser(username)
        if(response.isSuccessful)
            response.body()
        else null
    } catch (e: Exception) {
        println(e.message ?: "Unknown error")
        null
    }
}

val listOfUser = mutableListOf<User>()
val listUserDto = mutableListOf<UserDto>()

fun searchUsersFunction() = runBlocking {
    val response = searchUsers(mapOf("q" to "josh", "per_page" to 10))
    
    response?.items?.forEach {
        val userDtoResponse = it.login?.let { username ->
            getUser(username)
        }
        if (userDtoResponse != null) {
            listUserDto.add(userDtoResponse)
        }
    }
    
    listOfUser.addAll(listUserDto.toListUser())
    
    println(response?.items?.size.toString())
    println()
    println("result dto: ${response.toString()}")
    println()
    println("result model: ${listOfUser.toString()}")
}

searchUsersFunction()

10

result dto: UserResponse(incompleteResults=false, items=[UserDto(id=137, avatarUrl=https://avatars.githubusercontent.com/u/137?v=4, name=null, login=josh, bio=null, followers=null, following=null, location=null, email=null), UserDto(id=54473, avatarUrl=https://avatars.githubusercontent.com/u/54473?v=4, name=null, login=joshlong, bio=null, followers=null, following=null, location=null, email=null), UserDto(id=6692932, avatarUrl=https://avatars.githubusercontent.com/u/6692932?v=4, name=null, login=joshwcomeau, bio=null, followers=null, following=null, location=null, email=null), UserDto(id=13760, avatarUrl=https://avatars.githubusercontent.com/u/13760?v=4, name=null, login=joshaber, bio=null, followers=null, following=null, location=null, email=null), UserDto(id=10632, avatarUrl=https://avatars.githubusercontent.com/u/10632?v=4, name=null, login=goatslacker, bio=null, followers=null, following=null, location=null, email=null), UserDto(id=3272748, avatarUrl=https://avatars.githubuserc

In [84]:
for (i in 0..9) {
    println(
        """ val fakeUser${i+1} = User(
        id = ${listOfUser[i].id},
        avatarUrl = "${listOfUser[i].avatarUrl}",
        name = "${listOfUser[i].name}",
        login = "${listOfUser[i].login}",
        bio = "${listOfUser[i].bio.trim().take(50).replace("\r\n", " ")}...",
        followers = ${listOfUser[i].followers},
        following = ${listOfUser[i].following},
        location = "${listOfUser[i].location}",
        email = "${listOfUser[i].email}"
    )""")
}

 val fakeUser1 = User(
        id = 137,
        avatarUrl = "https://avatars.githubusercontent.com/u/137?v=4",
        name = "Joshua Peek",
        login = "josh",
        bio = "Biography not available...",
        followers = 2351,
        following = 184,
        location = "San Francisco, CA",
        email = "N/A"
    )
 val fakeUser2 = User(
        id = 54473,
        avatarUrl = "https://avatars.githubusercontent.com/u/54473?v=4",
        name = "Josh Long",
        login = "joshlong",
        bio = "Josh (@starbuxman) is the Spring Developer Advocat...",
        followers = 7955,
        following = 65,
        location = "San Francisco, CA",
        email = "N/A"
    )
 val fakeUser3 = User(
        id = 6692932,
        avatarUrl = "https://avatars.githubusercontent.com/u/6692932?v=4",
        name = "Joshua Comeau",
        login = "joshwcomeau",
        bio = "Software developer. Makes stuff....",
        followers = 3574,
        following = 25,
        location = "Mont